## Kemal Demirel: 191104091

## Kadir Çağan Eren: 

In [ ]:
import cv2
import numpy as np 
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
def traverse_directory(path, isItTrain):
    dataset = []
    labels = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if(file_path.endswith(".jpg")):
                img = cv2.imread(file_path)
                dataset.append(img)
                tmp_path = dirpath
                if(isItTrain):
                    label = tmp_path[tmp_path.find("train/")+len("train/"):]    
                else:
                    label = tmp_path[tmp_path.find("test/")+len("test/"):]    
                labels.append(label)
    return np.array(dataset), np.array(labels)

In [ ]:
X_train, y_train = traverse_directory(os.getcwd()+"/dataset/train", True)


In [ ]:
X_test, y_test = traverse_directory(os.getcwd()+"/dataset/test", False)


In [ ]:
def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (64, 64))
    return image

In [ ]:
for i in range(len(X_train)):
    X_train[i] = preprocess_image(X_train[i])

In [ ]:
for i in range(len(X_test)):
    X_test[i] = preprocess_image(X_test[i])


In [33]:
def feature_extraction_with_sift(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    max_descriptors = 50  # or any other value that is large enough to accomodate all the descriptors
    if descriptors is not None and descriptors.shape[0] > max_descriptors:
        descriptors = descriptors[:max_descriptors, :]
    elif descriptors is not None and descriptors.shape[0] < max_descriptors:
        descriptors = np.pad(descriptors, ((0, max_descriptors - descriptors.shape[0]), (0, 0)), 'constant')
    if descriptors is None:
        descriptors = np.zeros((max_descriptors, 128))
    return descriptors.flatten()

In [35]:
X_train_features = []
for image in X_train:
    features = feature_extraction_with_sift(image)
    if features is not None:
        X_train_features.append(features)
X_train_features = np.array(X_train_features, dtype='object')

In [36]:
X_test_features = []
for image in X_test:
    features = feature_extraction_with_sift(image)
    if features is not None:
        X_test_features.append(features)
X_test_features = np.array(X_test_features, dtype='object')

In [ ]:
def train_classifier(X_train, y_train):
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    return svm

In [ ]:
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

In [42]:
def predict(classifier, image):
    feature = feature_extraction_with_sift(image)
    prediction = classifier.predict([feature])
    return prediction[0]

In [37]:
classifier = train_classifier(X_train_features, y_train)

In [38]:
accuracy, precision, recall, f1 = evaluate_classifier(classifier, X_test_features, y_test)


0.9724407536907536


In [40]:
print(accuracy)

0.9670658682634731


In [44]:

new_image = cv2.imread("/Users/kemaldemirel/Desktop/Lectures/Bil 468/Project/Computer-Vision-Fruit-Vegetables-Detection/dataset/test/bell pepper/Image_2.jpg")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (64, 64))
prediction = predict(classifier, new_image)
print(prediction)

bell pepper
